In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
from utils import *
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [11]:
filters_names = ['~jump1-jump2-foot1-foot2-~isnew-~small_capq-small_volume-~opentop1-2','~jump1-jump2-foot1-foot2-~isnew-~small_capq-small_volume-~opentop1-2']

In [9]:
filter1 = getFilter("small_volume-yz0-~yz1-top1-opentop1") & ~yz2
filter2 = getFilter("~foot1-small_volume-yz0") & ~yz2

In [3]:
filter2 = minute & ~isnew & ~foot(2,1.0) & ~jump2 & foot(2,0.9) & minute2 & break_top & (data_df["open2"] < data_df["close1"] * 1.03) & (data_df["minute_low"] > data_df["close1"] * 0.97)

In [4]:
valid_df = data_df[filter2]
valid_df = valid_df.sort_values("minute")
valid_df = valid_df.groupby("date").first()
result(valid_df)

            mean  count
recent                 
1.0     0.978097      7
2.0     1.000510     62
3.0     1.024253     24
4.0     1.025986      9
5.0     0.982353      2
0.566037735849 106 1.00614486267


/Users/sgcy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [10]:
valid_df = data_df[filter1]
valid_df = valid_df.sort_values("minute")
valid_df = valid_df.groupby("date").first()
result(valid_df)

            mean  count
recent                 
3.0     1.048720     11
4.0     1.027445     14
5.0     1.027683      8
6.0     1.053568      3
7.0     1.026833     10
8.0     0.979451      4
10.0    0.936462      2
12.0    0.852262      2
15.0    1.099226      2
0.683333333333 60 1.02424183217


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [13]:
valid_df = data_df[filter2 & ~filter1]
valid_df = valid_df.sort_values("minute")
valid_df = valid_df.groupby("date").first()
result(valid_df)

            mean  count
recent                 
1.0     1.015925     27
3.0     1.028110     25
4.0     1.029435     18
5.0     0.892822      5
6.0     1.019253      6
7.0     0.991831      3
9.0     1.048907      2
10.0    1.029696      4
15.0    0.994086      2
0.616161616162 99 1.01708057118


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [8]:
valid_df

,change,last_day,stock,close0,high0,high_limit0,low0,open0,volume0,day0,...,day3,top_count,recent,minute_low,minute_volume,minute,st,circap,isnew,capq
date,,,,,,,,,,,,,,,,,,,,,
2007-01-12 00:00:00,1.018260,2007-01-11 00:00:00,600059.XSHG,7.70,7.81,8.30,7.32,7.51,8133193.0,4.0,...,7.0,13.0,2.0,8.29,3805507.0,09:56:00,False,1258.0,0.0,0.379487
2007-03-22 00:00:00,1.019589,2007-03-21 00:00:00,600714.XSHG,8.44,8.44,8.44,7.60,7.63,5132062.0,46.0,...,49.0,5.0,3.0,9.27,1536593.0,09:44:00,False,231.5,0.0,0.069834
2007-04-03 00:00:00,0.968254,2007-04-02 00:00:00,600340.XSHG,1.04,1.08,1.16,0.99,1.01,54535976.0,56.0,...,59.0,4.0,2.0,1.14,48762733.0,10:19:00,False,365.0,0.0,0.110106
2007-05-16 00:00:00,0.893488,2007-05-15 00:00:00,600864.XSHG,14.97,14.97,14.97,13.71,14.97,32190513.0,78.0,...,81.0,8.0,8.0,16.08,8292317.0,09:46:00,False,1169.0,0.0,0.352640
2007-05-21 00:00:00,1.011749,2007-05-18 00:00:00,000546.XSHE,12.66,13.09,13.48,12.20,12.48,12497587.0,79.0,...,82.0,9.0,2.0,13.65,3888000.0,09:49:00,False,715.0,0.0,0.215686
2007-06-14 00:00:00,1.095571,2007-06-13 00:00:00,600277.XSHG,3.55,3.64,3.97,3.28,3.62,17301119.0,102.0,...,105.0,6.0,2.0,3.79,18369469.0,10:15:00,False,399.0,0.0,0.120362
2007-06-20 00:00:00,1.004866,2007-06-19 00:00:00,000415.XSHE,3.40,3.40,3.40,3.10,3.12,52719990.0,104.0,...,107.0,4.0,3.0,3.62,14932958.0,09:53:00,False,653.0,0.0,0.196983
2007-08-01 00:00:00,0.930000,2007-07-30 00:00:00,002016.XSHE,4.96,5.03,5.54,4.84,4.95,5844659.0,114.0,...,117.0,4.0,2.0,5.42,4178436.0,10:01:00,False,360.5,0.0,0.108748
2007-08-08 00:00:00,1.013619,2007-08-07 00:00:00,002047.XSHE,4.25,4.27,4.52,4.08,4.15,12104145.0,140.0,...,143.0,4.0,2.0,4.59,31275203.0,10:22:00,False,336.0,0.0,0.101357
